In [1]:
NEWS_API_KEY = "c07df8265f264b20a0c2646a1740193e"
import requests
import json

import pandas as pd
from pandas.io.json import json_normalize
pd.options.display.max_rows = 10
import re
import nltk.sentiment.util

In [2]:
def get_all_articles(topic, date):
    page_number = 1
    all_articles = pd.DataFrame({})
    
    # TODO how do we make sure to get a couple articles per day?
    
    while page_number <= 10:
        url = ('https://newsapi.org/v2/everything?'
           'q=%s&'
           'from=%s&'
           'page=%d&'
           'language=en&'  # language is english
           'pageSize=100&'  # maximum page size
           'sortBy=popularity&'
           'apiKey=%s'
        ) % (topic, date, page_number, NEWS_API_KEY)
        response = requests.get(url)
        news = json.loads(response.text)
        if not news["articles"]:
            break
        all_articles = pd.concat([all_articles, json_normalize(news["articles"])], ignore_index=True)
        page_number += 1
        print(len(all_articles))
    return all_articles

def clean_articles(df):
    df["Date"] = df.publishedAt.str.split("T").str[0]
    # Do more text cleaning here
    #     df["description"] = df.description.str.replace('[^a-z A-Z]', '').str.lower().astype(str)
    df.dropna(subset=['description'], inplace=True)
    return df

In [3]:
import sys
from io import StringIO

def get_sentiment(articles):
    s = StringIO()
    sys.stdout = s
    articles["description"].apply(nltk.sentiment.util.demo_liu_hu_lexicon)
    # WHY ON EARTH WOULD YOU WRITE A FUNCTION THAT PRINTS INSTEAD OF RETURNING?
    sys.stdout = sys.__stdout__
    return s.getvalue().splitlines()

def get_sentiment_by_date(articles):
    return articles.groupby(["Date","sentiment"]).size().unstack(fill_value=0)

In [4]:
articles = get_all_articles("apple", "2004-06-11")

100
200
300
400
500
600
700
800
900
1000


In [5]:
articles = clean_articles(articles)

In [6]:
articles["sentiment"] = get_sentiment(articles)

In [72]:
sentiment_by_date = get_sentiment_by_date(articles)
sentiment_by_date.columns

Index(['Negative', 'Neutral', 'Positive'], dtype='object', name='sentiment')

In [73]:
import pickle
pickle.dump(sentiment_by_date, open("sentiment_by_date.pkl", "wb"))

In [45]:
date1 = "2013-06-11"
date2 = "2018-01-11"
url = ('https://newsapi.org/v2/everything?'
           'q=%s&'
           'from=%s&'
           'to=%s&'
           'page=%d&'
           'language=en&'  # language is english
           'pageSize=10&'  # maximum page size
           'apiKey=%s'
        ) % ("apple", date1, date2, 1, NEWS_API_KEY)
response = requests.get(url)
# news = json.loads(response.text)
response.text

'{"status":"ok","totalResults":160233,"articles":[{"source":{"id":null,"name":"Glassdoor.com"},"author":null,"title":"Apple Apple Retail - Expert says \\"It’s Apple.\\"","description":"From Apple Apple Retail - Expert(Current Employee)&dash; Rating 5 out of 5 — Tue, 17 Oct 2017 Pros Great culture! The management is great, products are fun, and you learn so many life skills. There aren’t many companies like Apple out there. Cons Most employe…","url":"https://www.glassdoor.com/Reviews/Employee-Review-Apple-RVW17411967.htm","urlToImage":"https://media.glassdoor.com/lst2x/1138/apple-office.jpg","publishedAt":"2017-10-18T04:27:05Z"},{"source":{"id":null,"name":"Glassdoor.com"},"author":null,"title":"Apple Specialist says \\"Apple is Apple\\"","description":"From Apple Specialist(Current Employee)&dash; Rating 4 out of 5 — Tue, 9 Jan 2018 Pros Great pay for retail so you can’t complain about that. Hands down the best job I’ve ever had that was considered retail. Cons The cons are definitely 